In [ ]:
import requests

csv_data_dl = requests.get(
    "https://raw.githubusercontent.com/microsoft/msticpy/main/tests/testdata/az_net_flows.csv"
)
with open("network_traffic.csv", "wb") as f:
    f.write(csv_data_dl.content)


In [ ]:
import pandas as pd

network_traffic_df = pd.read_csv("network_traffic.csv")
network_traffic_df.head()


In [ ]:
network_traffic_df.iloc[0].T


In [ ]:
ips = network_traffic_df["AllExtIPs"].unique()
ips


In [ ]:
import os

otx_api_key = os.environ["OTX_KEY"]
ip = ips[0]
url = f"https://otx.alienvault.com/api/v1/indicators/IPv4/{ip}/general"
headers = {"X-OTX-API-KEY": otx_api_key}
otx_response = requests.get(url, headers=headers)
otx_response.status_code


In [ ]:
otx_response.json()


In [ ]:
otx_series = pd.Series(otx_response.json())
otx_series[["reputation", "pulse_info", "asn", "city", "country_code"]]


In [ ]:
pulses_df = pd.DataFrame(otx_series["pulse_info"]["pulses"])
pulses_df


In [ ]:
def row_style(row):
    if row.related_indicator_is_active == 1:
        return pd.Series("background-color: red", row.index)
    else:
        return pd.Series("background-color: none", row.index)


pulses_df.style.apply(row_style, axis=1)


In [ ]:
network_traffic_df[network_traffic_df["AllExtIPs"] == ips[0]].plot(
    y="TotalAllowedFlows", x="TimeGenerated"
)
